<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/MovieDBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [62]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [63]:
df_netflix=pd.read_csv('/content/titles_net.csv')

In [64]:
df_amazon=pd.read_csv('/content/titles_amazon.csv')

In [65]:
df_hbo=pd.read_csv('/content/titles_hbo.csv')

In [66]:
df=pd.concat([df_netflix,df_amazon,df_hbo])

In [67]:
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [68]:
df.columns

Index(['id', 'title', 'type', 'description', 'release_year',
       'age_certification', 'runtime', 'genres', 'production_countries',
       'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity',
       'tmdb_score'],
      dtype='object')

In [69]:
df_movies=df.drop_duplicates()
df_movies.duplicated().sum()

0

In [70]:
df_movies.drop(['description','age_certification'],axis=1,inplace=True)

In [71]:
df_movies['production_countries']

,production_countries
0,['US']
1,['US']
2,['US']
3,['GB']
4,"['GB', 'US']"
...,...
3289,['PR']
3290,['PA']
3291,[]
3292,[]


In [72]:
df_movies['production_countries']=df_movies['production_countries'].str.replace(r"\[",'',regex=True).str.replace(r"'",'',regex=True).str.replace(r"\]",'',regex=True)

In [73]:
df_movies['lead_production_countries']=df_movies['production_countries'].str.split(',').str[0]

In [74]:
df_movies['lead_production_countries']

,lead_production_countries
0,US
1,US
2,US
3,GB
4,GB
...,...
3289,PR
3290,PA
3291,
3292,


In [75]:
df_movies['prod_countries_cnt']=df_movies['production_countries'].str.split(',').str.len()

In [76]:
df_movies['prod_countries_cnt']

,prod_countries_cnt
0,1
1,1
2,1
3,1
4,2
...,...
3289,1
3290,1
3291,1
3292,1


In [77]:
df_movies['lead_production_countries']

,lead_production_countries
0,US
1,US
2,US
3,GB
4,GB
...,...
3289,PR
3290,PA
3291,
3292,


In [78]:
df_movies['lead_production_countries']=df_movies['lead_production_countries'].replace('',np.nan)

In [79]:
df_movies['lead_production_countries']

,lead_production_countries
0,US
1,US
2,US
3,GB
4,GB
...,...
3289,PR
3290,PA
3291,NaN
3292,NaN


In [80]:
df_movies['genres']

,genres
0,['documentation']
1,"['drama', 'crime']"
2,"['drama', 'action', 'thriller', 'european']"
3,"['fantasy', 'action', 'comedy']"
4,"['war', 'action']"
...,...
3289,"['romance', 'music']"
3290,['comedy']
3291,['comedy']
3292,['comedy']


In [81]:
df_movies['genres']=df_movies['genres'].str.replace(r"\[",'',regex=True).str.replace(r"'",'',regex=True).str.replace(r"\]",'',regex=True)
df_movies['main_genre']=df_movies['genres'].str.split(',').str[0]
df_movies['main_genre']=df_movies['main_genre'].replace('',np.nan)

In [82]:
df_movies['main_genre']

,main_genre
0,documentation
1,drama
2,drama
3,fantasy
4,war
...,...
3289,romance
3290,comedy
3291,comedy
3292,comedy


In [83]:
df_movies['genres']

,genres
0,documentation
1,"drama, crime"
2,"drama, action, thriller, european"
3,"fantasy, action, comedy"
4,"war, action"
...,...
3289,"romance, music"
3290,comedy
3291,comedy
3292,comedy


In [84]:
df_movies.drop(['genres','production_countries'],axis=1,inplace=True)

In [85]:
df_movies.columns

Index(['id', 'title', 'type', 'release_year', 'runtime', 'seasons', 'imdb_id',
       'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score',
       'lead_production_countries', 'prod_countries_cnt', 'main_genre'],
      dtype='object')

In [86]:
df_movies.shape

(18980, 14)

In [87]:
df_movies.isnull().sum()

,0
id,0
title,1
type,0
release_year,0
runtime,0
seasons,14772
imdb_id,1394
imdb_score,1873
imdb_votes,1910
tmdb_popularity,670


In [88]:
df_movies.dropna(inplace=True)

In [89]:
df_movies.set_index('title',inplace=True)

df_movies.drop(['id','imdb_id'],axis=1,inplace=True)

In [90]:
df_movies.head()

,type,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,lead_production_countries,prod_countries_cnt,main_genre
title,,,,,,,,,,,
Monty Python's Flying Circus,SHOW,1969,30,4.0,8.8,73424.0,17.617,8.306,GB,1,comedy
Seinfeld,SHOW,1989,24,9.0,8.9,308824.0,130.213,8.301,US,1,comedy
Knight Rider,SHOW,1982,51,4.0,6.9,34115.0,50.267,7.500,US,1,scifi
Thomas & Friends,SHOW,1984,10,24.0,6.5,5104.0,42.196,6.500,GB,1,animation
Saved by the Bell,SHOW,1989,23,5.0,7.1,35034.0,19.855,8.000,US,1,family


In [91]:
# Encoding Categorical Features

In [94]:
dummies=pd.get_dummies(df_movies[['type','lead_production_countries','main_genre']], drop_first=True)
dummies.astype(int)

,lead_production_countries_AR,lead_production_countries_AT,lead_production_countries_AU,lead_production_countries_BE,lead_production_countries_BR,lead_production_countries_CA,lead_production_countries_CH,lead_production_countries_CL,lead_production_countries_CN,lead_production_countries_CO,...,main_genre_history,main_genre_horror,main_genre_music,main_genre_reality,main_genre_romance,main_genre_scifi,main_genre_sport,main_genre_thriller,main_genre_war,main_genre_western
title,,,,,,,,,,,,,,,,,,,,,
Monty Python's Flying Circus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Seinfeld,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Knight Rider,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Thomas & Friends,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Saved by the Bell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Level Playing Field,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Os Ausentes,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Through Our Eyes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
df_movies_dum=pd.concat([df_movies,dummies],axis=1)

# drop original categorical columns after creating dummy variables
df_movies_dum.drop(['type','lead_production_countries','main_genre'],axis=1,inplace=True)

In [98]:
df_movies_dum

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,prod_countries_cnt,lead_production_countries_AR,lead_production_countries_AT,...,main_genre_history,main_genre_horror,main_genre_music,main_genre_reality,main_genre_romance,main_genre_scifi,main_genre_sport,main_genre_thriller,main_genre_war,main_genre_western
title,,,,,,,,,,,,,,,,,,,,,
Monty Python's Flying Circus,1969,30,4.0,8.8,73424.0,17.617,8.306,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Seinfeld,1989,24,9.0,8.9,308824.0,130.213,8.301,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Knight Rider,1982,51,4.0,6.9,34115.0,50.267,7.500,1,False,False,...,False,False,False,False,False,True,False,False,False,False
Thomas & Friends,1984,10,24.0,6.5,5104.0,42.196,6.500,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Saved by the Bell,1989,23,5.0,7.1,35034.0,19.855,8.000,1,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Level Playing Field,2021,26,1.0,5.5,60.0,4.595,5.000,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Os Ausentes,2021,46,1.0,5.9,59.0,4.624,10.000,1,False,False,...,False,False,False,False,False,False,False,False,False,False
Through Our Eyes,2021,33,1.0,6.1,38.0,0.840,1.000,1,False,False,...,False,False,False,False,False,False,False,False,False,False


In [99]:
#Scaling MinMaxScaler
#apply minmaxscaler to scale the data for model training
scaler=MinMaxScaler()
df_scaled=scaler.fit_transform(df_movies_dum)
df_scaled=pd.DataFrame(df_scaled,columns=df_movies_dum.columns)

df_scaled

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,prod_countries_cnt,lead_production_countries_AR,lead_production_countries_AT,...,main_genre_history,main_genre_horror,main_genre_music,main_genre_reality,main_genre_romance,main_genre_scifi,main_genre_sport,main_genre_thriller,main_genre_war,main_genre_western
0,0.397727,0.168539,0.058824,0.9125,0.037009,0.007913,0.815870,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.625000,0.134831,0.156863,0.9250,0.155671,0.058490,0.815326,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.545455,0.286517,0.058824,0.6750,0.017194,0.022579,0.728261,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.568182,0.056180,0.450980,0.6250,0.002570,0.018954,0.619565,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.625000,0.129213,0.078431,0.7000,0.017658,0.008919,0.782609,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,0.988636,0.146067,0.000000,0.5000,0.000028,0.002064,0.456522,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3290,0.988636,0.258427,0.000000,0.5500,0.000027,0.002077,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3291,0.988636,0.185393,0.000000,0.5750,0.000017,0.000377,0.021739,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3292,0.988636,0.191011,0.019608,0.3125,0.000067,0.001158,0.510870,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
# DBSCAN
eps_array=[0.2,0.5,1]
min_samples_array=[5,10,30]

In [104]:
for eps in eps_array:
  for min_samples in min_samples_array:
    cluster = DBSCAN(eps=eps, min_samples=min_samples).fit(df_scaled)
    cluster_labels = cluster.labels_

    # Skip iteration if all points are in one cluster
    if len(set(cluster_labels)) == 1:
      continue

    silhouette_avg = silhouette_score(df_scaled, cluster_labels)

    # Get the number of clusters (excluding noise points labeled as -1)
    n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

    print('For eps=', eps, "For min samples=", min_samples, "Number of clusters=", n_clusters, "The average Silhouette score is:", silhouette_avg)

For eps= 0.2 For min samples= 5 Number of clusters= 74 The average Silhouette score is: 0.4378840737098286
For eps= 0.2 For min samples= 10 Number of clusters= 36 The average Silhouette score is: 0.36601440046646755
For eps= 0.2 For min samples= 30 Number of clusters= 16 The average Silhouette score is: 0.23106054247198202
For eps= 0.5 For min samples= 5 Number of clusters= 90 The average Silhouette score is: 0.6019560501740351
For eps= 0.5 For min samples= 10 Number of clusters= 55 The average Silhouette score is: 0.5303679432698052
For eps= 0.5 For min samples= 30 Number of clusters= 20 The average Silhouette score is: 0.36228604161700484
For eps= 1 For min samples= 5 Number of clusters= 92 The average Silhouette score is: 0.6091664186394289
For eps= 1 For min samples= 10 Number of clusters= 56 The average Silhouette score is: 0.5362809971937993
For eps= 1 For min samples= 30 Number of clusters= 21 The average Silhouette score is: 0.37121300388037515


In [105]:
dbscan_model=DBSCAN(eps=1,min_samples=5).fit(df_scaled)

print("Clusters",len(set(dbscan_model.labels_)))
print("Silhouette Score",silhouette_score(df_scaled,dbscan_model.labels_))

Clusters 93
Silhouette Score 0.6091664186394289


Save Clusters for Recommendation

In [106]:
df_movies['dbscan_clusters']=dbscan_model.labels_

In [107]:
df_movies

,type,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,lead_production_countries,prod_countries_cnt,main_genre,dbscan_clusters
title,,,,,,,,,,,,
Monty Python's Flying Circus,SHOW,1969,30,4.0,8.8,73424.0,17.617,8.306,GB,1,comedy,0
Seinfeld,SHOW,1989,24,9.0,8.9,308824.0,130.213,8.301,US,1,comedy,1
Knight Rider,SHOW,1982,51,4.0,6.9,34115.0,50.267,7.500,US,1,scifi,2
Thomas & Friends,SHOW,1984,10,24.0,6.5,5104.0,42.196,6.500,GB,1,animation,3
Saved by the Bell,SHOW,1989,23,5.0,7.1,35034.0,19.855,8.000,US,1,family,4
...,...,...,...,...,...,...,...,...,...,...,...,...
Level Playing Field,SHOW,2021,26,1.0,5.5,60.0,4.595,5.000,US,1,documentation,35
Os Ausentes,SHOW,2021,46,1.0,5.9,59.0,4.624,10.000,BR,1,action,-1
Through Our Eyes,SHOW,2021,33,1.0,6.1,38.0,0.840,1.000,US,1,documentation,35


In [111]:
import random

def recommend_movie(movie_name:str):
  movie_name=movie_name.lower()
  df_movies['name']=df_movies.index.str.lower()

  movie=df_movies[df_movies['name'].str.contains(movie_name,na=False)]

  if not movie.empty:
    cluster=movie['dbscan_clusters'].values[0]
    cluster_movies=df_movies[df_movies['dbscan_clusters']==cluster]

    if len(cluster_movies)>=5:
      recommended_movies=random.sample(list(cluster_movies.index),5)
    else:
      recommended_movies=list(cluster_movies.index)
    print("We can recommend you these movies:")
    for m in recommended_movies:
      print(m)
  else:
    print("Movie not found in the dataset")

In [113]:
input_movie=input("Enter movie name: ")
recommend_movie(input_movie)

Enter movie name: Kid vs. Kat
We can recommend you these movies:
Xiaolin Chronicles
Tales from the Cryptkeeper
Di-Gata Defenders
Creeped Out
Dark Matter


In [115]:
input_movie=input("Enter movie name: ")
recommend_movie(input_movie)

Enter movie name: Travelers
We can recommend you these movies:
Class of the Titans
Creeped Out
Xiaolin Chronicles
Todd and the Book of Pure Evil
Travelers


In [116]:
df_movies.to_csv("clustered_movies.csv",index=False)